[View in Colaboratory](https://colab.research.google.com/github/hlecuanda/zconf/blob/master/Calendar_Correct.ipynb)

In [0]:
try:
    from oauth2client.service_account import ServiceAccountCredentials
    from google.oauth2 import service_account
    import gspread
except:
    !pip -q install google-auth google-auth-httplib2 google-api-python-client gspread
    from oauth2client.service_account import ServiceAccountCredentials
    import gspread
    # from gspread.models import Cell
finally:
    import re
    import sys
    import httplib2
    import datetime
    from apiclient import discovery
    from google.colab import files
    from google.colab import output
    from google.colab import widgets

#TODO: pull encrypted keyfile from gcs
#TODO: decrypt keyfile using cloud KMS API
#TODO: marlk procedure as shareable snippet for project
SERVICE_ACCOUNT_FILE = 'colab-notebooks-7b143dd37c90.json'

SCOPES = ['https://www.googleapis.com/auth/calendar',
          'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file',
          'https://www.googleapis.com/auth/drive.metadata.readonly',
          'https://www.googleapis.com/auth/spreadsheets']

try:
    with open(SERVICE_ACCOUNT_FILE, "r") as f:
        print()
except:
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
                name=fn, length=len(uploaded[fn])))
finally:
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

delegated_credentials = credentials.create_delegated('hector@lecuanda.com')  
delegated_http = delegated_credentials.authorize(httplib2.Http())
drive    = discovery.build('drive', 'v3', http=delegated_http)
sheets   = discovery.build('sheets','v4', http=delegated_http)
24calendar = discovery.build('calendar','v3', http=delegated_http)

In [0]:
CAL_ID='lecuanda.com_o04mf6ehur85udttarcuuuqno8@group.calendar.google.com'
CUTOFF=utc=datetime.datetime(2018,5,20,0,0,0,0)
evts=[]

#evts.append(cal.get('items',[]))
#allfields=['kind','etag','id','status','htmlLink','created','updated','summary',
#           'creator','organizer','start','end','iCalUID','sequence','reminders']
wanted=['start','end','summary','description','status']
extended=['start','end','summary','number','type','duration','status']
page_token = None
while True:
    cal=calendar.events().list(calendarId=CAL_ID,pageToken=page_token).execute()
    for event in cal['items']:
        if event['status']=='cancelled':
            break
        e={}
        for wantedField in wanted:
            try:
                e[wantedField]=event[wantedField]
            except KeyError:
                e[wantedField]='N/A'
                
        evts.append(e)
    page_token = cal.get('nextPageToken')
    if not page_token:
        break

def renderEvent(no,field,events=evts):
    retval=None
    selfie=' * '
    record_type = re.compile('incoming|outgoing|missed')
    duration= re.compile('\d{2}:\d{2}:\d{2}')
    number = re.compile('Number:\s.\d+')
    
    p=re.compile('(?P<number>\d+).+(?P<type>outgoing|incoming|missed).+\n.+(?P<duration>\d{2}:\d{2}:\d{2})')
    q=re.compile('(?P<number>\d+).+(?P<type>missed).+\n')

    if field in ['start', 'end']:
        if isinstance(events[no][field],str):
            pass
        elif 'dateTime' in events[no][field].keys():
            interim = events[no][field]['dateTime']
            retval ='{} {}'.format(interim[0:10],interim[11:19])
        elif 'date' in events[no][field].keys():
            interim = events[no][field]['date']
            retval= '{} {}'.format(interim,'00:00:00')
            
    elif field in ['summary']:
        fv=events[no][field]
        if 'Called' in fv:
            return  fv.replace('Called ','')
        elif 'Missed' in fv:
            return fv.replace('Missed call from ','')
        elif 'Call from' in fv:
            return fv.replace('Call from ','')

    elif field in ['number','type','duration']:
        fv=events[no]['description']
        m=p.search(fv)
        n=q.search(fv)
        if m:
            vector = {'number':    m.group('number'),
                       'type':     m.group('type'),
                       'duration': m.group('duration')}
        elif n:
            vector ={'number':     n.group('number'),
                     'type':       n.group('type'),
                     'duration':  '00:00:00'} 
        else:
            vector ={'number':     events[no]['description'],
                     'type':       'not a call',
                     'duration':  '00:00:00'}
            
        return vector[field] 
            
    elif field in ['creator','organizer']:
        if isinstance(events[no][field],str):
            pass
        elif 'self' in events[no][field].keys():
            retval = events[no][field]['displayName'] + selfie
        else:
            retval = events[no][field]['displayName']
    if not retval: 
        retval = events[no][field]
    return retval

widgets.create_grid(range(0,12),extended,
                    lambda x,y : renderEvent(x,y),
                    header_column=True,
                   )

In [0]:
gc = gspread.authorize(delegated_credentials)
Cell = gspread.models.Cell
shid = '1YLHRgCsUPkFG1wwfuZ0nP70J3Ebg2l2LB_zJiZchcc4'

try:
    sh = gc.open_by_key(shid)
except:
    sh = gc.create('Lupita Call Log')
    shid = sh.id
finally:
    print(sh)

worksheet = sh.sheet1
print(worksheet)

def loadSpreadsheet(eventsList,fieldsList,wks):
    dataset=[]
    r=0
    for row in eventsList:
        for col in fieldsList:
            c = fieldsList.index(col)
            cell = Cell(r+1,c+1)
            cell.value=renderEvent(r,col)
            dataset.append(cell)
        r=r+1    
    wks.update_cells(dataset,'USER_ENTERED') 
    
loadSpreadsheet(evts,extended,worksheet)